In [2]:
import glob
import math
import os
import random
import shutil
import subprocess
from pathlib import Path

import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
from tqdm import tqdm
import torch.nn.functional as F

# Layers
Định nghĩa một số layers cho yolov4

In [3]:
def make_divisible(v, divisor):
    # Đảm bảo tất cả các layer có số kênh chia hết cho 8
    return math.ceil(v/divisor) * divisor

In [4]:
class Flatten(nn.Module):
    # Sử dụng sau nn.AdaptiveAvgPool12d(1) để xóa bỏ 2 chiều sau
    def forward(self, x):
        return x.view(x.size(0), -1)

In [5]:
class Concat(nn.Module):
    # Nối 1 list các tensor dọ theo các chiều của chúng
    def __init__(self, dismension=1):
        super(Concat, self).__init__()
        self.d = dismension

    def forward(self, x):
        return torch.cat(x, self.d)

In [6]:
class FeatureConcat(nn.Module):
    def __init__(self, layers):
        super(FeatureConcat, self).__init__()
        self.layers = layers
        self.mutiple = len(layers) > 1

    def forward(self, x, outputs):
        return torch.cat([outputs[i] for i in self.layers], 1) if self.mutiple else outputs[self.layers[0]]

In [7]:
class WeightedFeatureFusion(nn.Module):
    # weighted sum of 2 or more layers https://arxiv.org/abs/1911.09070
    def __init__(self, layers, weight=False):
        super(WeightedFeatureFusion, self).__init__()
        self.layers = layers
        self.weight = weight
        self.n = len(layers) + 1
        if weight:
            self.w = nn.Parameter(torch.zeros(self.n), requires_grad=True)
    def forward(self, x, outputs):
        # Weights
        if self.weight:
            w = torch.sigmoid(self.w)*(2/self.n)
            x = x * w[0]
        
        # Fusion (Hợp nhất)
        nx = x.shape[1] # Số kênh input
        for i in range(self.n-1):
            a = outputs[self.layers[i] * w[i-1] if self.weight else outputs[self.layers[i]]] # thêm các feature
            na = a.shape[1] # feature channels

            # Adjust channels (Điều chỉnh kênh)
            if nx == na: # Giống nhau về chiêu
                x = x + a
            elif nx > na: # slice input
                x[:, :na] = x[:,:na] +  a
                # hoac a = nn.ZeroPad2d((0,0,0,0,dc))(a); x = x + a
            else: # slide feature
                x = x + a[:, :nx]

        return x